### imports for data-science

In [1]:
import pandas as pd
import json
import glob
import logging
import time
import os
from tqdm import tqdm

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


### how many calls a single worker can take

### workers 
- there are 755 workers for this firm
- 

In [2]:

# Load workers data and format correctly
logging.info("Loading workers data and calculating recommendation scores...")
workers_df = pd.read_json('extracted/workers.json').transpose().reset_index()
workers_df.columns = ['worker_id', 'name', 'base_salary']
workers_df.sort_values(by='base_salary')

2024-11-13 21:49:16,120 - INFO - Loading workers data and calculating recommendation scores...


,worker_id,name,base_salary
2,w_653b3a89-c5fa-466b-a477-f0c09a724cdd,Douglas Case,8259
289,w_f483de89-4555-480b-9c8a-3fbd5fb29d5e,Joe Carr,8414
624,w_5959f088-3f09-4685-8a92-b2b8bbf38bf6,Michael Erickson,8512
47,w_c5b9e1d1-0d90-4ecf-b33c-1aed6e792f65,Mary Flint,8588
161,w_1c85fb3c-075b-425f-b910-36d7d7bb43fe,Maggie Meader,8790
...,...,...,...
613,w_006be806-a7bb-4a8e-b08b-3a00a117f15a,Bonnie Thrall,12582
585,w_d31fdc3a-f933-4f71-83d2-6bf75f79b7fc,Helen Goldizen,12590
518,w_8cdb33b2-e637-42ca-a34a-a8c73915f466,Stella Bell,12907
44,w_3898999d-649f-461e-9b90-f15eaa0b3550,Norman London,13529


### location of the worker
 - the worker data doenst include data, but the calls have location. so using this we can find where the worker is 
 - go through the schedules and for every worker check where the location is
 - if true set the worker to that location


In [18]:
calls_data = {}
for file_path in glob.glob('Week3/calls/*.json'):
    with open(file_path, 'r') as file:
        file_data = json.load(file
                              )
        for location, calls in file_data.items():
            if location in calls_data:
                calls_data[location].update(calls)  # Merge calls for this location
            else:
                calls_data[location] = calls  # Initialize calls for this location


In [20]:
# Process the calls data into a DataFrame for easier handling
calls_list = []
for location, calls in calls_data.items():
    for call_id, call_details in calls.items():
        call_details['call_id'] = call_id
        call_details['location'] = location  # Add location to each call record
        calls_list.append(call_details)

previous_calls_df = pd.DataFrame(calls_list)
previous_calls_df

,date,tlf_number,technical_problem,difficulty,commission,call_id,location
0,1970-01-01 00:00:00+00:00,30134029,basic_hardware_troubleshooting,hard,864.0,c_3d390740-68df-4579-b437-fcd1a25b0ca8,bangalore
1,1970-01-01 00:00:00+00:00,60496609,software_installation_and_configuration,hard,900.0,c_80affc8f-bf2c-4c0f-9460-9c95b2e007fa,bangalore
2,1970-01-01 00:00:00+00:00,91681812,operating_system_support,hard,882.0,c_b3278b67-74c0-42d0-9007-5b62e2f0c72e,bangalore
3,1970-01-01 00:00:00+00:00,04938996,zoom_problems,hard,834.0,c_d93e9286-1775-4e83-9711-17fa8011be6d,bangalore
4,1970-01-01 00:00:00+00:00,59536017,software_installation_and_configuration,easy,600.0,c_a7598d3e-8046-4fff-91e5-9b3eb6c52ed2,bangalore
...,...,...,...,...,...,...,...
260052,1970-01-01 00:00:00+00:00,60822405,account_and_security_issues,medium,710.0,c_089dc1ec-ca09-4d9c-82eb-9a4a1802fe23,hyderabad
260053,1970-01-01 00:00:00+00:00,62836494,operating_system_support,easy,588.0,c_984b60ae-efed-4533-b6b2-4dcaf9cf551b,hyderabad
260054,1970-01-01 00:00:00+00:00,17220135,teams_problems,easy,532.0,c_2e9a9bef-b496-4c12-9cdb-9646547d0954,hyderabad
260055,1970-01-01 00:00:00+00:00,47242825,zoom_problems,hard,834.0,c_cd0f66f1-52df-40f4-b1d2-5d26b92183f6,hyderabad


In [5]:

# Load the schedules data (worker-to-call assignments)
previous_schedule_data = {}
for file_path in glob.glob('Week2/call_schedule_Uke2/*.json'):
    with open(file_path, 'r') as file:
        previous_schedule_data.update(json.load(file))  # Merge data from multiple schedule files

# Create a dictionary for call location lookup for faster access
call_location_dict = previous_calls_df.set_index('call_id')['location'].to_dict()

# Map each worker to their location based on the calls they were assigned
worker_locations = {}

for worker_id, assigned_calls in previous_schedule_data.items():
    worker_call_locations = set()  # Use set to ensure unique locations
    for call_id in assigned_calls:
        call_location = call_location_dict.get(call_id)  # Fast lookup for the call location
        if call_location:
            worker_call_locations.add(call_location)
    
    # If the worker has calls from more than one location, we assign "mixed" or other label
    if len(worker_call_locations) == 1:
        worker_locations[worker_id] = worker_call_locations.pop()
    else:
        worker_locations[worker_id] = "mixed"  # Multiple locations for simplicity

# Ensure worker_locations is added to workers_df
workers_df['location'] = workers_df['worker_id'].map(worker_locations)

### check if there any worker in more than one office
- should NOT be the case

In [6]:
# # Print workers with 'mixed' locations
# mixed_location_workers = workers_df[workers_df['location'] == 'mixed']
# mixed_location_workers


In [7]:
# Load and flatten reports data for worker performance
reports_records = []
for file_path in glob.glob('Week2/call_report_week2/*.json'):
    reports_records.extend(pd.read_json(file_path).to_dict(orient='records'))
reports_df = pd.DataFrame(reports_records)

reports_df = reports_df.drop(['professional_score'], axis=1)


In [8]:
# Merge workers_df with reports_df on 'worker_id' to include worker details in the report data
merged_df = pd.merge(reports_df, workers_df, on='worker_id', how='left')
# Perform the merge with specified suffixes
merged_df = pd.merge(merged_df, previous_calls_df, on='call_id', how='left', suffixes=('', '_previous'))
# If you want only one location column, drop the duplicate
merged_df = merged_df.drop(columns=['location_previous'])

In [9]:
# Count the number of unique workers who worked at each location
location_worker_counts = merged_df.groupby('location')['worker_id'].nunique()

# Display the result
print(location_worker_counts)


location
bangalore     77
hyderabad    104
mumbai       574
Name: worker_id, dtype: int64


In [10]:
worker_call_counts = merged_df.groupby(['worker_id', 'location']).size().reset_index(name='call_count')

# Step 2: Calculate the average calls per worker at each location
avg_calls_per_worker_at_location = worker_call_counts.groupby('location')['call_count'].mean()

In [11]:
# 1. Merge `reports_df` with `previous_calls_df` on `call_id` to get the technical problem for each call
merged_df = pd.merge(reports_df, previous_calls_df[['call_id', 'technical_problem']], on='call_id', how='left')

# 2. Calculate the average call time for each worker by technical problem
avg_call_time_by_problem = merged_df.groupby(['worker_id', 'technical_problem'])['call_time'].mean().reset_index()

# 3. Pivot the data to have each technical problem as a separate column for average call time
avg_call_time_pivot = avg_call_time_by_problem.pivot(index='worker_id', columns='technical_problem', values='call_time')

# 4. Rename columns to reflect the technical problem type and 'avg_call_time' prefix
avg_call_time_pivot.columns = [f'avg_{col}_call_time' for col in avg_call_time_pivot.columns]

# 5. Calculate the overall average call time for each worker
overall_avg_call_time = merged_df.groupby('worker_id')['call_time'].mean().reset_index()
overall_avg_call_time.columns = ['worker_id', 'avg_overall_call_time']

# Calculate average recommendation score for each worker
worker_performance = merged_df.groupby('worker_id')['likely_to_recommend'].mean().reset_index()
worker_performance.columns = ['worker_id', 'avg_recommendation_score']

# 6. Calculate Profit Per Call (PPT) for each worker by technical problem
# Group by `worker_id` and `technical_problem` to get total profit and count of calls
profit_per_call = merged_df.groupby(['worker_id', 'technical_problem']).agg(
    total_profit=('call_profit', 'sum'),
    call_time=('call_time', 'sum'),
    
).reset_index()

# Calculate PPT by dividing total profit by the call count for each technical problem and worker
profit_per_call['PPT'] = profit_per_call['total_profit'] / profit_per_call['call_time']

# Pivot `PPT` data to have each technical problem as a separate column
ppt_pivot = profit_per_call.pivot(index='worker_id', columns='technical_problem', values='PPT')
ppt_pivot.columns = [f'PPT_{col}' for col in ppt_pivot.columns]

# 7. Merge all calculated metrics into `workers_df`
workers_with_avg_call_time = pd.merge(workers_df, avg_call_time_pivot, on='worker_id', how='left')
workers_with_avg_call_time = pd.merge(workers_with_avg_call_time, overall_avg_call_time, on='worker_id', how='left')
workers_with_avg_call_time = pd.merge(workers_with_avg_call_time, worker_performance, on='worker_id', how='left')
workers_with_avg_call_time = pd.merge(workers_with_avg_call_time, ppt_pivot, on='worker_id', how='left')


 - merge the worker with average times back to reports:

In [12]:
# Merge workers_df with reports_df on 'worker_id' to include worker details in the report data
merged_df = pd.merge(reports_df, workers_with_avg_call_time, on='worker_id', how='left')

## top performers

In [13]:
def find_top_performers(technical_problem):
    # Filter out relevant columns for the technical problem and `PPT`
    filtered_df = merged_df[['worker_id', f'PPT_{technical_problem}', 'call_profit', 'call_time', 'likely_to_recommend', 'avg_overall_call_time', 'location', 'name']]

    # Sort within each worker by PPT in descending order and take the top 30 PPT values for each worker
    top_ppt_values = (
        filtered_df.sort_values(by=[f'PPT_{technical_problem}'], ascending=False)
        .groupby('worker_id')
        .head(30)
    )

    # Group by worker_id and calculate metrics, including the mean of top 30 PPT values
    top_performers = top_ppt_values.groupby('worker_id').agg({
        'name': 'first',
        'call_profit': 'sum',
        'call_time': 'mean',
        'likely_to_recommend': 'mean',
        'avg_overall_call_time': 'mean',
        f'PPT_{technical_problem}': 'mean',  # This now only uses the top 30 PPT values
        'location': 'first'
    }).reset_index()

    # Sort by mean PPT for the technical problem in descending order to get top performers
    top_performers = top_performers.sort_values(by=f'PPT_{technical_problem}', ascending=False)

    return top_performers


# test getting top performer

In [14]:


avg_columns = [
    "account_and_security_issues",
    "basic_hardware_troubleshooting",
    "cloud_and_storage_solutions",
    "device_and_peripheral_setup",
    "email_related_issues",
    "internet_problems",
    "operating_system_support",
    "software_installation_and_configuration",
    "teams_problems",
    "zoom_problems"
]

# # Set display options to prevent truncation
# pd.set_option('display.max_rows', None)  # Show all rows in output
# pd.set_option('display.max_colwidth', None)  # Show full column width without truncation

for x in tqdm(avg_columns):
    top_performers = find_top_performers(x)
    print("Top performer for", x, "is\n", top_performers.head(), "\n\n\n\n\n")

# # Optionally, reset options to default afterward
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_colwidth')

 50%|█████     | 5/10 [00:00<00:00, 20.77it/s]

Top performer for account_and_security_issues is
                                   worker_id            name  call_profit  \
378  w_82eaec0a-e3b1-4346-a093-b39ab1d6de74     James James        22016   
52   w_10a80ae7-f569-430f-b2cc-5b86011409a3     Bonnie Wise        20359   
29   w_0a13a0de-8759-4adb-9955-c8aec755ae15    Brian Temple        21230   
752  w_fe89bc24-7071-4e47-89cd-f4b180817d55  Michael Holmes        20739   
621  w_d5213d16-8ea1-421d-8ddc-35d377944891   Tory Fountain        21705   

     call_time  likely_to_recommend  avg_overall_call_time  \
378  24.793002             1.686667              23.876621   
52   24.734522             1.533333              24.535216   
29   20.530333             1.453333              22.258479   
752  26.021632             2.286667              27.742191   
621  23.589067             1.416667              22.988027   

     PPT_account_and_security_issues   location  
378                        49.381851     mumbai  
52                  

100%|██████████| 10/10 [00:00<00:00, 21.35it/s]

Top performer for internet_problems is
                                   worker_id          name  call_profit  \
694  w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803   Efren Selva        20976   
487  w_aab97eef-a107-404f-819f-e504de5f0513  Paul Livings        21576   
717  w_f275ce75-072d-47cd-a29a-50ce0c3f1a01   Ted Foreman        20314   
200  w_49fb706d-fb94-41d8-9032-e32886bc65a5   Corine Damm        20409   
691  w_ea2668e4-cafc-4e4f-a381-21fd5b9255e2  Amos Labonte        20383   

     call_time  likely_to_recommend  avg_overall_call_time  \
694  23.523651             1.450000              26.040043   
487  25.071094             2.726667              24.874812   
717  30.773482             1.173333              31.232615   
200  24.594392             1.326667              26.388121   
691  27.861887             2.603333              31.053779   

     PPT_internet_problems   location  
694              44.655831     mumbai  
487              42.994958     mumbai  
717              41.21

- find call time sum
- find average for all technical problems call time for that specific worker
- asign amount of calls depending on (call time sum/technical problem average call time)
- this does not find the best worker for that technical problem

### group the calls by the workers id to get the average performance of the worker
this is tecnically ehat the top performer function does

In [15]:
# Group by 'worker_id' and aggregate the necessary columns
worker_report = merged_df.groupby('worker_id').agg({
    'avg_recommendation_score': 'mean',  # Get the average recommendation score
    # Add other columns you want to aggregate (e.g., 'call_profit', 'location', etc.)
    'call_profit': 'sum',  # Example: Sum of call profits for the worker
    'location': 'first',  # Example: First location for the worker
    'name': 'first',  # Example: First name for the worker (assuming it's a single name)
}).reset_index()

In [16]:
def calculate_priority_order(technical_issues):
    # Create a list to store the average PPT for the top 30 workers of each technical problem
    technical_problem_priority = []

    for technical_problem in technical_issues:
        # Get top 30 performers for the technical problem
        top_performers = find_top_performers(technical_problem).head(30)
        
        # Calculate the average PPT for these top 30 performers
        avg_ppt_top_30 = top_performers[f'PPT_{technical_problem}'].mean()
        
        # Append to list with technical problem name and calculated average PPT
        technical_problem_priority.append({
            'technical_problem': technical_problem,
            'avg_ppt_top_30': avg_ppt_top_30
        })

    # Convert to DataFrame for easier sorting
    priority_df = pd.DataFrame(technical_problem_priority)

    # Sort technical problems by average PPT in descending order
    priority_df = priority_df.sort_values(by='avg_ppt_top_30', ascending=False).reset_index(drop=True)

    return priority_df

# List of technical issues
avg_columns = [
    "account_and_security_issues",
    "basic_hardware_troubleshooting",
    "cloud_and_storage_solutions",
    "device_and_peripheral_setup",
    "email_related_issues",
    "internet_problems",
    "operating_system_support",
    "software_installation_and_configuration",
    "teams_problems",
    "zoom_problems"
]

# Get the order in which calls for each technical problem should be assigned
priority_order_df = calculate_priority_order(avg_columns)
print("Priority order of technical problems for call assignment:\n", priority_order_df)


Priority order of technical problems for call assignment:
                          technical_problem  avg_ppt_top_30
0                 operating_system_support       43.476213
1  software_installation_and_configuration       41.763669
2              account_and_security_issues       40.372246
3              cloud_and_storage_solutions       39.432123
4                            zoom_problems       38.712044
5              device_and_peripheral_setup       38.579499
6           basic_hardware_troubleshooting       38.312316
7                     email_related_issues       37.022446
8                           teams_problems       36.373472
9                        internet_problems       36.088688


In [17]:
import os
import json
import logging
import time
import pandas as pd
from tqdm import tqdm

MAX_CALL_LIMIT_PER_WORKER = 80
MIN_CALL_LIMIT_PER_WORKER = 25  # Set to -1 for unlimited calls

# Define function to assign calls to workers based on performance and difficulty, in priority order
def assign_calls_to_workers_in_priority_order(calls_df, workers_df, priority_order_df):
    # Initialize the schedule dictionary to store call assignments for each worker
    schedule = {worker_id: [] for worker_id in workers_df['worker_id']}
    
    # Filter calls by difficulty level to prioritize hard calls first
    calls_by_difficulty = {
        'hard': calls_df[calls_df['difficulty'] == 'hard'],
        'medium': calls_df[calls_df['difficulty'] == 'medium'],
        'easy': calls_df[calls_df['difficulty'] == 'easy']
    }

    # Go through each technical problem in the priority order
    for _, problem_row in priority_order_df.iterrows():
        technical_problem = problem_row['technical_problem']
        
        # Prioritize hard, medium, and then easy calls within the technical problem
        for difficulty, calls in calls_by_difficulty.items():
            # print(f"Assigning {difficulty} calls for {technical_problem}")

            # Filter calls for the current technical problem
            calls_for_problem = calls[calls['technical_problem'] == technical_problem]

            # Assign calls to the best-performing workers in the correct location
            for _, call in tqdm(calls_for_problem.iterrows(), desc=f"Assigning {difficulty} calls for {technical_problem}"):
                call_id = call['call_id']
                location = call['location']
                
                # Get top performers for the technical problem, filtered by location
                top_performers = find_top_performers(technical_problem)
                location_top_performers = top_performers[top_performers['location'] == location]
                
                assigned = False
                # Attempt to assign the call to a top-performing worker in the correct location
                for _, worker in location_top_performers.iterrows():
                    worker_id = worker['worker_id']
                    # Ensure worker's call limit is within the defined min and max limits
                    if (MIN_CALL_LIMIT_PER_WORKER == -1 or len(schedule[worker_id]) >= MIN_CALL_LIMIT_PER_WORKER) and \
                       (MAX_CALL_LIMIT_PER_WORKER == -1 or len(schedule[worker_id]) < MAX_CALL_LIMIT_PER_WORKER):
                        schedule[worker_id].append(call_id)
                        assigned = True
                        break

                # If no top performer is available, assign to any random worker in the location
                if not assigned:
                    available_workers = workers_df[workers_df['location'] == location]
                    if not available_workers.empty:
                        random_worker = available_workers.sample(n=1).iloc[0]
                        worker_id = random_worker['worker_id']
                        # Check if the worker is within the call limits
                        if (MIN_CALL_LIMIT_PER_WORKER == -1 or len(schedule[worker_id]) >= MIN_CALL_LIMIT_PER_WORKER) and \
                           (MAX_CALL_LIMIT_PER_WORKER == -1 or len(schedule[worker_id]) < MAX_CALL_LIMIT_PER_WORKER):
                            schedule[worker_id].append(call_id)
                        else:
                            # logging.warning(f"Call {call_id} could not be assigned to any worker in {location}")
                            pass
                    
    return schedule

# Process each feature call file in the directory using os.listdir() instead of glob
calls_directory = 'Week3/calls/'

# Loop through all files in the directory
for file_name in tqdm(os.listdir(calls_directory)):
    # Process only JSON files
    if file_name.endswith('.json'):
        file_path = os.path.join(calls_directory, file_name)
        start_time = time.time()
        
        # logging.info(f"Processing feature calls file: {file_path}")
        
        # Load and flatten feature calls data for the current file
        feature_records = []
        with open(file_path) as f:
            data = json.load(f)
        
        for location, calls in data.items():
            # logging.info(f"Processing future calls in: {location}")
            for call_id, call_info in calls.items():
                call_info['call_id'] = call_id
                call_info['location'] = location
                feature_records.append(call_info)
        
        feature_calls_df = pd.DataFrame(feature_records)

        # Generate the call schedule for this file
        call_schedule = assign_calls_to_workers_in_priority_order(feature_calls_df, worker_report, priority_order_df)

        # Convert call_schedule to output format
        output_schedule = {worker_id: calls for worker_id, calls in call_schedule.items() if calls}
        output_file = f'call_schedule_{file_name}'

        # Save the schedule to a JSON file
        output_file_path = os.path.join(calls_directory, output_file)
        with open(output_file_path, 'w') as outfile:
            json.dump(output_schedule, outfile, indent=4)
        
        total_time = time.time() - start_time
        # logging.info(f"Call schedule for {file_name} generated and saved in {total_time:.2f} seconds.")


Assigning hard calls for operating_system_support: 811it [01:09, 11.69it/s]
Assigning medium calls for operating_system_support: 791it [00:48, 16.41it/s]
Assigning easy calls for operating_system_support: 802it [00:42, 18.89it/s]
Assigning hard calls for software_installation_and_configuration: 787it [00:59, 13.12it/s]
Assigning medium calls for software_installation_and_configuration: 799it [01:28,  9.06it/s]
Assigning easy calls for software_installation_and_configuration: 799it [00:57, 13.80it/s]
Assigning hard calls for account_and_security_issues: 808it [01:01, 13.22it/s]
Assigning medium calls for account_and_security_issues: 799it [01:03, 12.59it/s]
Assigning easy calls for account_and_security_issues: 774it [00:45, 17.17it/s]
Assigning hard calls for cloud_and_storage_solutions: 779it [00:43, 17.76it/s]
Assigning medium calls for cloud_and_storage_solutions: 755it [00:39, 18.94it/s]
Assigning easy calls for cloud_and_storage_solutions: 844it [00:42, 19.83it/s]
Assigning hard ca

KeyboardInterrupt: 